# Load times from SWMA and SWMA Catalogue

In [1]:
import pandas as pd

# Load the GOFC data more efficiently
SWMA_dir = "/home/dschwarz/Documents/MT/Dataset/Dataset_MSc/SWMA_RDCDFI.csv"
GFOC_data = pd.read_csv(SWMA_dir, low_memory=True)

# Load the SWMA data more efficiently
SWMA_dir = "/home/dschwarz/Documents/MT/Dataset/Dataset_MSc/SWMA_RDAWFI.csv"
SWMA_data = pd.read_csv(SWMA_dir, low_memory=True)

In [ ]:
# Time conversion
SWMA_time = SWMA_data['time']
SWMA_time = pd.to_datetime(SWMA_time, format='%Y-%m-%d %H:%M:%S')
GFOC_start = SWMA_time[0]
SWMA_end = SWMA_time[len(SWMA_time) - 1]
SWMA_time = SWMA_data['time']
SWMA_time = pd.to_datetime(SWMA_time, format='%Y-%m-%d %H:%M:%S')
SWMA_start = SWMA_time[0]
SWMA_end = SWMA_time[len(SWMA_time) - 1]

# Load Shock & both ICME catalogues

In [ ]:
import pandas as pd

# import the data
Shock = pd.read_csv('./Dataset_IPshocks/shocks_20250514_121541.csv')

# Shock_Wind = Shock[Shock['Spacecraft'] == 'Wind']
# Shock_Wind = Shock[Shock['Spacecraft'] == 'ACE']
# Shock_Wind = Shock[Shock['Spacecraft'] == 'DSCOVR']
# Shock_Wind = Shock[Shock['Spacecraft'] == 'OMNI']

Helios_Wind = pd.read_csv('./Dataset_ICMECAT/helio4cast_icmecat.csv')

RC = pd.read_csv('./Dataset_ICMECAT/RC_icmecat_dates.csv')

# Create Flags

### First I want to check if all the times are in the correct order

In [ ]:
# ======================================================================================
# Event times ordered from the latest to the earliest
# ======================================================================================

# event_time_dt = pd.DatetimeIndex(Shock['Time'])
# event_time_dt = pd.DatetimeIndex(Helios_Wind['icme_start_time'])

# for i in range(len(event_time_dt) - 1):
#     if event_time_dt[-i - 2] < event_time_dt[-i -1]:
#         print("The times are not in the correct order")
#         print(i - 1)
#         print(event_time_dt[-i - 2])
#         print(event_time_dt[-i - 1])
#         break
# print("The times are in the correct order")

# ======================================================================================
# Event times ordered from the earliest to the latest
# ======================================================================================

event_time_dt = pd.DatetimeIndex(RC['ICME_Start'])

for i in range(len(event_time_dt) - 1):
    if event_time_dt[-i - 2] > event_time_dt[-i -1]:
        print("The times are not in the correct order")
        print(i - 1)
        print(event_time_dt[-i - 2])
        print(event_time_dt[-i - 1])
        break
print("The times are in the correct order")

The times are in the correct order


The times are in the correct order. However, I need to think about the order.

## Creating Flags for SWMA

- for ICME events I need to consider the start and end time of the event --> 1 if between start and end and 0 if not
- for Shock events there is only one time

In [ ]:
import numpy as np
from datetime import timedelta

# only select times that are in the range of the GFOC data
Shock_GFOC = Shock[pd.to_datetime(Shock['Time'], format='%Y-%m-%d %H:%M:%S') >= GFOC_start]
Shock_GFOC = Shock_GFOC[pd.to_datetime(Shock_GFOC['Time'], format='%Y-%m-%d %H:%M:%S') <= GFOC_end]
Helios_GFOC = Helios_Wind[pd.to_datetime(Helios_Wind['mo_end_time'], format='%Y-%m-%d %H:%M:%S') >= GFOC_start]
Helios_GFOC = Helios_GFOC[pd.to_datetime(Helios_GFOC['icme_start_time'], format='%Y-%m-%d %H:%M:%S') <= GFOC_end]
RC_GFOC = RC[pd.to_datetime(RC['ICME_End'], format='%Y-%m-%d %H:%M:%S') >= GFOC_start]
RC_GFOC = RC_GFOC[pd.to_datetime(RC_GFOC['Disturbance_time'], format='%Y-%m-%d %H:%M:%S') <= GFOC_end]

Shock_flag_GFOC = np.zeros(len(GFOC_time), dtype=int)
Helios_shock_flag_GFOC = np.zeros(len(GFOC_time), dtype=int)
Helios_mostart_flag_GFOC = np.zeros(len(GFOC_time), dtype=int)
Helios_moend_flag_GFOC = np.zeros(len(GFOC_time), dtype=int)
RC_dist_flag_GFOC = np.zeros(len(GFOC_time), dtype=int)
RC_start_flag_GFOC = np.zeros(len(GFOC_time), dtype=int)
RC_end_flag_GFOC = np.zeros(len(GFOC_time), dtype=int)

# print(GFOC_time[:10])

# Loop through Shock events, define a ±15 second window
for _, row in Shock_GFOC.iterrows():
    center = pd.to_datetime(row['Time'])
    start = center - timedelta(seconds=15)
    end = center + timedelta(seconds=15)
    
    # Set Shock flag to 1 for all GFOC_time values in this 30-second window
    Shock_flag_GFOC[(GFOC_time >= start) & (GFOC_time <= end)] = 1

# Loop through Shock events, define a ±15 second window
for _, row in Helios_GFOC.iterrows():
    center = pd.to_datetime(row['icme_start_time'])
    start = center - timedelta(seconds=15)
    end = center + timedelta(seconds=15)
    
    # Set Shock flag to 1 for all GFOC_time values in this 30-second window
    Helios_shock_flag_GFOC[(GFOC_time >= start) & (GFOC_time <= end)] = 1

# Loop through each start-end interval in Helios_GFOC
for _, row in Helios_GFOC.iterrows():
    center = pd.to_datetime(row['mo_start_time'])
    start = center - timedelta(seconds=15)
    end = center + timedelta(seconds=15)
    
    # Set flag to 1 where GFOC_time is within this interval
    Helios_mostart_flag_GFOC[(GFOC_time >= start) & (GFOC_time <= end)] = 1

    center = pd.to_datetime(row['mo_end_time'])
    start = center - timedelta(seconds=15)
    end = center + timedelta(seconds=15)
    
    # Set flag to 1 where GFOC_time is within this interval
    Helios_moend_flag_GFOC[(GFOC_time >= start) & (GFOC_time <= end)] = 1


# Loop through each start-end interval in RC_GFOC
for _, row in RC_GFOC.iterrows():
    center = pd.to_datetime(row['Disturbance_time'])
    # print(center)
    start = center - timedelta(seconds=15)
    end = center + timedelta(seconds=15)
    
    # Set Shock flag to 1 for all GFOC_time values in this 30-second window
    RC_dist_flag_GFOC[(GFOC_time >= start) & (GFOC_time <= end)] = 1

for _, row in RC_GFOC.iterrows():
    center = pd.to_datetime(row['ICME_Start'])
    start = center - timedelta(seconds=15)
    end = center + timedelta(seconds=15)
    
    # Set flag to 1 where GFOC_time is within this interval
    RC_start_flag_GFOC[(GFOC_time >= start) & (GFOC_time <= end)] = 1

    center = pd.to_datetime(row['ICME_End'])
    start = center - timedelta(seconds=15)
    end = center + timedelta(seconds=15)
    
    # Set flag to 1 where GFOC_time is within this interval
    RC_end_flag_GFOC[(GFOC_time >= start) & (GFOC_time <= end)] = 1


GFOC_flag_df = pd.DataFrame({
    'GFOC_time': GFOC_time,
    'IPShock': Shock_flag_GFOC,
    'Helios_Shock': Helios_shock_flag_GFOC,
    'Helios_MoStart': Helios_mostart_flag_GFOC,
    'Helios_MoEnd': Helios_moend_flag_GFOC,
    'RC_dist': RC_dist_flag_GFOC,
    'RC_Start': RC_start_flag_GFOC,
    'RC_End': RC_end_flag_GFOC
})

# Save the DataFrame to a CSV file
GFOC_flag_df.to_csv('GFOC_flags.csv', index=False)

2023-01-03 21:00:00
2023-02-27 10:00:00
2023-03-01 00:00:00
2023-03-02 09:00:00
2023-03-12 06:00:00
2023-03-15 04:27:00
2023-03-23 05:00:00
2023-04-18 14:02:00
2023-04-23 17:38:00
2023-05-09 22:50:00
2023-05-12 06:33:00
2023-05-20 10:00:00
2023-07-07 12:00:00
2023-07-14 16:11:00
2023-07-16 19:22:00
2023-07-25 22:33:00
2023-08-05 02:55:00
2023-09-02 04:34:00
2023-09-17 02:34:00
2023-09-18 13:31:00
2023-09-24 20:43:00
2023-09-28 21:31:00
2023-10-05 22:25:00
2023-11-12 06:13:00
2023-12-01 09:26:00
2023-12-15 11:48:00
2023-12-17 07:39:00
2024-01-03 15:10:00
2024-01-23 23:00:00
2024-01-26 20:00:00
2024-03-21 02:50:00
2024-03-24 14:34:00
2024-04-30 12:22:00
2024-05-02 14:07:00
2024-05-10 17:05:00
2024-05-12 09:17:00
2024-05-17 13:29:00
2024-06-28 10:14:00


In [32]:
# print(GFOC_flag_df.loc[GFOC_flag_df['RC_dist'] == 1, 'GFOC_time'])

center = pd.to_datetime('2024-06-28 10:14:00')
# center = pd.to_datetime('2024-06-29 04:00:00')
start = center - timedelta(seconds=15*1e3*2)
end = center + timedelta(seconds=15*1e3*4)

print(start, end)
    
# Set flag to 1 where GFOC_time is within this interval
print(GFOC_flag_df.loc[(GFOC_time >= start) & (GFOC_time <= end), 'GFOC_time'])

2024-06-28 01:54:00 2024-06-29 02:54:00
1481759   2024-06-29 00:00:00
1481760   2024-06-29 00:00:30
1481761   2024-06-29 00:01:00
1481762   2024-06-29 00:01:30
1481763   2024-06-29 00:02:00
                  ...        
1482103   2024-06-29 02:52:00
1482104   2024-06-29 02:52:30
1482105   2024-06-29 02:53:00
1482106   2024-06-29 02:53:30
1482107   2024-06-29 02:54:00
Name: GFOC_time, Length: 349, dtype: datetime64[ns]


## Creating Flags for SWMA

In [33]:
import numpy as np
from datetime import timedelta

# only select times that are in the range of the SWMA data
Shock_SWMA = Shock[pd.to_datetime(Shock['Time'], format='%Y-%m-%d %H:%M:%S') >= SWMA_start]
Shock_SWMA = Shock_SWMA[pd.to_datetime(Shock_SWMA['Time'], format='%Y-%m-%d %H:%M:%S') <= SWMA_end]
Helios_SWMA = Helios_Wind[pd.to_datetime(Helios_Wind['mo_end_time'], format='%Y-%m-%d %H:%M:%S') >= SWMA_start]
Helios_SWMA = Helios_SWMA[pd.to_datetime(Helios_SWMA['icme_start_time'], format='%Y-%m-%d %H:%M:%S') <= SWMA_end]
RC_SWMA = RC[pd.to_datetime(RC['ICME_End'], format='%Y-%m-%d %H:%M:%S') >= SWMA_start]
RC_SWMA = RC_SWMA[pd.to_datetime(RC_SWMA['Disturbance_time'], format='%Y-%m-%d %H:%M:%S') <= SWMA_end]

Shock_flag_SWMA = np.zeros(len(SWMA_time), dtype=int)
Helios_shock_flag_SWMA = np.zeros(len(SWMA_time), dtype=int)
Helios_mostart_flag_SWMA = np.zeros(len(SWMA_time), dtype=int)
Helios_moend_flag_SWMA = np.zeros(len(SWMA_time), dtype=int)
RC_dist_flag_SWMA = np.zeros(len(SWMA_time), dtype=int)
RC_start_flag_SWMA = np.zeros(len(SWMA_time), dtype=int)
RC_end_flag_SWMA = np.zeros(len(SWMA_time), dtype=int)

# print(SWMA_time[:10])

# Loop through Shock events, define a ±15 second window
for _, row in Shock_SWMA.iterrows():
    center = pd.to_datetime(row['Time'])
    start = center - timedelta(seconds=15)
    end = center + timedelta(seconds=15)
    
    # Set Shock flag to 1 for all SWMA_time values in this 30-second window
    Shock_flag_SWMA[(SWMA_time >= start) & (SWMA_time <= end)] = 1

# Loop through Shock events, define a ±15 second window
for _, row in Helios_SWMA.iterrows():
    center = pd.to_datetime(row['icme_start_time'])
    start = center - timedelta(seconds=15)
    end = center + timedelta(seconds=15)
    
    # Set Shock flag to 1 for all SWMA_time values in this 30-second window
    Helios_shock_flag_SWMA[(SWMA_time >= start) & (SWMA_time <= end)] = 1

# Loop through each start-end interval in Helios_SWMA
for _, row in Helios_SWMA.iterrows():
    center = pd.to_datetime(row['mo_start_time'])
    start = center - timedelta(seconds=15)
    end = center + timedelta(seconds=15)
    
    # Set flag to 1 where SWMA_time is within this interval
    Helios_mostart_flag_SWMA[(SWMA_time >= start) & (SWMA_time <= end)] = 1

    center = pd.to_datetime(row['mo_end_time'])
    start = center - timedelta(seconds=15)
    end = center + timedelta(seconds=15)
    
    # Set flag to 1 where SWMA_time is within this interval
    Helios_moend_flag_SWMA[(SWMA_time >= start) & (SWMA_time <= end)] = 1


# Loop through each start-end interval in RC_SWMA
for _, row in RC_SWMA.iterrows():
    center = pd.to_datetime(row['Disturbance_time'])
    # print(center)
    start = center - timedelta(seconds=15)
    end = center + timedelta(seconds=15)
    
    # Set Shock flag to 1 for all SWMA_time values in this 30-second window
    RC_dist_flag_SWMA[(SWMA_time >= start) & (SWMA_time <= end)] = 1

for _, row in RC_SWMA.iterrows():
    center = pd.to_datetime(row['ICME_Start'])
    start = center - timedelta(seconds=15)
    end = center + timedelta(seconds=15)
    
    # Set flag to 1 where SWMA_time is within this interval
    RC_start_flag_SWMA[(SWMA_time >= start) & (SWMA_time <= end)] = 1

    center = pd.to_datetime(row['ICME_End'])
    start = center - timedelta(seconds=15)
    end = center + timedelta(seconds=15)
    
    # Set flag to 1 where SWMA_time is within this interval
    RC_end_flag_SWMA[(SWMA_time >= start) & (SWMA_time <= end)] = 1


SWMA_flag_df = pd.DataFrame({
    'SWMA_time': SWMA_time,
    'IPShock': Shock_flag_SWMA,
    'Helios_Shock': Helios_shock_flag_SWMA,
    'Helios_MoStart': Helios_mostart_flag_SWMA,
    'Helios_MoEnd': Helios_moend_flag_SWMA,
    'RC_dist': RC_dist_flag_SWMA,
    'RC_Start': RC_start_flag_SWMA,
    'RC_End': RC_end_flag_SWMA
})

# Save the DataFrame to a CSV file
SWMA_flag_df.to_csv('SWMA_flags.csv', index=False)